# Libraries

In [117]:
library(plyr)

library(tidyverse)
library(DT) # dependency
library(ggthemes) # dependency

In [118]:
source("tools.R") # Ad-hoc tools

# General variables and functions

In [119]:
se <- function(x) {
    sd(x)/sqrt(length(x))
}

bar_graph <- function(plot_data) {
    plot_data %>%
        ggplot(aes(x=x, y=y))+
        geom_bar(stat="identity", position=position_dodge()) +
        geom_errorbar(aes(ymin=lower_bound, ymax=upper_bound),
                      position=position_dodge(.9),
                      width=0.2 )
}

decimals <- 2 # for rounding

# Import data

In [120]:
directory <- "Data January 2017/"

dat <- get_fc_table(directory)

### Data mutation

In [121]:
# Change column names
colnames(dat) <- c( "row_id",
                    "left_door",
                    "right_door",
                    "rt",
                    "KEY_RESPONSE_TIME",
                    "DISPLAY_TIME", 
                    "key_pressed",
                    "block_level_1_fc",
                    "block",
                    "id",
                    "experiment" )


# Remove ".jpg" of the doors' names; Change {f,j} to {left, right}; Add chosen_door column
dat <- dat %>%
    mutate(left_door = substr(left_door, 0, 7), 
           right_door = substr(right_door, 0, 7),
           key_pressed = ifelse(key_pressed=='f', 'left', 'right'), 
           chosen_door = ifelse(key_pressed=='left', left_door, right_door)) 

Cue values

In [122]:
for(index in 1:length(dat$id)){
    dat$left_speed[index] <- get_cues(dat$experiment[index], dat$left_door[index])[1]
    dat$left_safety[index] <- get_cues(dat$experiment[index], dat$left_door[index])[2]
    
    dat$right_speed[index] <- get_cues(dat$experiment[index], dat$right_door[index])[1]
    dat$right_safety[index] <- get_cues(dat$experiment[index], dat$right_door[index])[2]
    
    dat$chosen_speed[index] <- get_cues(dat$experiment[index], dat$chosen_door[index])[1]
    dat$chosen_safety[index] <- get_cues(dat$experiment[index], dat$chosen_door[index])[2]
}

Add final columns

In [123]:
# Number of cues to check
dat <- dat %>%
    mutate(steps_ttb_speed = ifelse(left_speed == right_speed, 2, 1),
           steps_ttb_safety = ifelse(left_safety == right_safety, 2, 1),
           unknown_door_involved = (left_speed == 0 | right_speed ==0),
           chosen_door_is_unknown = (chosen_speed == 0) ) # Checking one cue is enough

In [124]:
# ASSUMPTION: when an unknown door is involved, the decision takes 1 step
dat[dat$unknown_door_involved==TRUE, ]$steps_ttb_speed <- 1
dat[dat$unknown_door_involved==TRUE, ]$steps_ttb_safety <- 1

Now that we have the cue values, we can calculate the **predicted** decisions

In [125]:
# Choice per TTB.
# ASSUMPTION: unknown door is never chosen

## TTB Speed

dat[!dat$unknown_door_involved & dat$left_speed > dat$right_speed, 'chosen_ttb_speed'] <- 'left'

dat[!dat$unknown_door_involved & dat$left_speed < dat$right_speed, 'chosen_ttb_speed'] <- 'right'

dat[!dat$unknown_door_involved & dat$left_speed == dat$right_speed & dat$left_safety > dat$right_safety, 'chosen_ttb_speed'] <- 'left'

dat[!dat$unknown_door_involved & dat$left_speed == dat$right_speed & dat$left_safety < dat$right_safety, 'chosen_ttb_speed'] <- 'right'

dat[dat$unknown_door_involved & dat$left_speed == 0 , 'chosen_ttb_speed'] <- 'right'

dat[dat$unknown_door_involved & dat$right_speed == 0 , 'chosen_ttb_speed'] <- 'left'


## TTB Safety

dat[!dat$unknown_door_involved & dat$left_safety > dat$right_safety, 'chosen_ttb_safety'] <- 'left'

dat[!dat$unknown_door_involved & dat$left_safety < dat$right_safety, 'chosen_ttb_safety'] <- 'right'

dat[!dat$unknown_door_involved & dat$left_safety == dat$right_safety & dat$left_speed > dat$right_speed, 'chosen_ttb_safety'] <- 'left'

dat[!dat$unknown_door_involved & dat$left_safety == dat$right_safety & dat$left_speed < dat$right_speed, 'chosen_ttb_safety'] <- 'right'

dat[dat$unknown_door_involved & dat$left_safety == 0 , 'chosen_ttb_safety'] <- 'right'

dat[dat$unknown_door_involved & dat$right_safety == 0 , 'chosen_ttb_safety'] <- 'left'

In [126]:
# When choices are equivalent, the predicted choice is unknown
dat$equivalent_options = FALSE

dat[dat$right_speed == dat$left_speed & dat$right_safety == dat$left_safety, 'equivalent_options'] <- TRUE

dat[dat$equivalent_options == TRUE, 'chosen_ttb_safety'] <- 'unknown'
dat[dat$equivalent_options == TRUE, 'chosen_ttb_speed'] <- 'unknown'

# Import ACT-R data

In [127]:
agents <- read_csv("../ACT-R/outputs/experiment-results.csv")

Parsed with column specification:
cols(
  id = col_integer(),
  opt1 = col_character(),
  opt2 = col_character(),
  response = col_character(),
  time = col_double(),
  threshold = col_integer()
)


In [128]:
# Response times
agents$rt <- agents$time - c(0, agents$time[1 : length(agents$time) - 1])
agents[agents$rt < 0, 'rt'] <- agents[agents$rt < 0, 'time']

# Chosen door
agents$chosen_door <- ifelse(agents$response == 'f', agents$opt1, agents$opt2)

#################

# Corrected TTB Safety (CSfM predictions)
agents$csfm <- "unknown"

# A is always superior
agents$csfm <- ifelse(agents$opt1 == "a" | agents$opt2 == "a", "a", agents$csfm)

# Option B
agents$csfm <- ifelse(agents$csfm == "unknown" & (agents$opt1 == "b" | agents$opt2 == "b"), "b", agents$csfm)

# C vs D, wins C
agents$csfm <- ifelse(agents$opt1 == "c" & agents$opt2 == "d", "c", agents$csfm)
agents$csfm <- ifelse(agents$opt1 == "d" & agents$opt2 == "c", "c", agents$csfm)

# C or D versus unknown
agents$csfm <- ifelse(agents$csfm == "unknown" & (agents$opt1 == "c" | agents$opt1 == "d"), agents$opt2, agents$csfm)
agents$csfm <- ifelse(agents$csfm == "unknown" & (agents$opt2 == "c" | agents$opt2 == "d"), agents$opt1, agents$csfm)

In [129]:
# Unknown door involved
agents$unknown_door_involved <- ( (agents$opt1 != 'a' & agents$opt1 != 'b' & agents$opt1 != 'c' & agents$opt1 != 'd') | 
                                  (agents$opt2 != 'a' & agents$opt2 != 'b' & agents$opt2 != 'c' & agents$opt2 != 'd') )

# Equivalent alternatives
agents$equivalent_options <- agents$csfm == "unknown"

# STEPS:
## Default is 1
agents$steps_ttb_safety <- 1

## 2 steps

### A vs B
agents$steps_ttb_safety <- ifelse((agents$opt1 == "a" & agents$opt2 == "b") |
                                  (agents$opt1 == "b" & agents$opt2 == "a"), 2, agents$steps_ttb_safety)

### C vs D
agents$steps_ttb_safety <- ifelse((agents$opt1 == "c" & agents$opt2 == "d") |
                                  (agents$opt1 == "d" & agents$opt2 == "c"), 2, agents$steps_ttb_safety)

<hr>
<hr>

# Accordance rates

Accordance rate is the proportion of human responses that were correctly predicted

In [130]:
accordance <- function( data, column ) {
    
    column <- enquo(column)
    
    data %>%
        mutate( correctly_predicted = (key_pressed == !!column) ) %>%
        group_by( id ) %>%
        summarize( accordance_rate = mean(correctly_predicted) ) %>%
        summarize( accordance_rate_mean = mean(accordance_rate), 
                   se = se(accordance_rate),
                   lower_bound = t.test(accordance_rate, conf.level=0.95)$conf.int[1],
                   upper_bound = t.test(accordance_rate, conf.level=0.95)$conf.int[2]) %>%
        round( decimals )
}

In [131]:
accordance_actr <- function( data, column ) {
    
    column <- enquo(column)
    
    data %>%
        mutate( correctly_predicted = (chosen_door == !!column) ) %>%
        group_by( id ) %>%
        summarize( accordance_rate = mean(correctly_predicted) ) %>%
        summarize( accordance_rate_mean = mean(accordance_rate), 
                   se = se(accordance_rate),
                   lower_bound = t.test(accordance_rate, conf.level=0.95)$conf.int[1],
                   upper_bound = t.test(accordance_rate, conf.level=0.95)$conf.int[2]) %>%
        round( decimals )
}

### 1. Cases where both models predict the same outcome

In [219]:
print('Safety')
dat %>% 
    filter( chosen_ttb_speed == chosen_ttb_safety ,
            unknown_door_involved == FALSE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_safety )

print('Speed')
dat %>% 
    filter( chosen_ttb_speed == chosen_ttb_safety ,
            unknown_door_involved == FALSE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_speed )

[1] "Safety"


accordance_rate_mean,se,lower_bound,upper_bound
0.92,0.01,0.89,0.95


[1] "Speed"


accordance_rate_mean,se,lower_bound,upper_bound
0.92,0.01,0.89,0.95


### 2. Cases where both models predict different outcomes

In [220]:
print('Safety')
dat %>% 
    filter( chosen_ttb_speed != chosen_ttb_safety ,
            unknown_door_involved == FALSE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_safety )

print('Speed')
dat %>% 
    filter( chosen_ttb_speed != chosen_ttb_safety ,
            unknown_door_involved == FALSE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_speed )

[1] "Safety"


accordance_rate_mean,se,lower_bound,upper_bound
0.96,0.02,0.92,1


[1] "Speed"


accordance_rate_mean,se,lower_bound,upper_bound
0.04,0.02,0,0.08


### 3. Both choices are equivalent

Rate of 'right' choices. It should be 50%.

In [221]:
dat %>%
    filter( equivalent_options == TRUE ) %>%
    mutate(selected_right = key_pressed == 'right') %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )

selected_right_rate_mean,se,lower_bound,upper_bound
0.55,0.03,0.5,0.6


### 4. Unknown door is involved

In [222]:
print('Safety')
dat %>% 
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_safety )

print('Speed')
dat %>% 
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_speed )

[1] "Safety"


accordance_rate_mean,se,lower_bound,upper_bound
0.58,0.02,0.53,0.62


[1] "Speed"


accordance_rate_mean,se,lower_bound,upper_bound
0.58,0.02,0.53,0.62


### 4.2 Unknown door is involved. Digging deeper

Here I check what happens when different types of doors are contrasted against an unknown door

In [223]:
print("Safety, Speed = [1, 1]")
safety_val = 1
speed_val = 1

dat %>%
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    filter( right_safety == safety_val | left_safety == safety_val,
            right_speed == speed_val | left_speed == speed_val) %>%
    mutate(selected_right = chosen_door_is_unknown == FALSE) %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )


print("Safety, Speed = [1, -1]")
safety_val = 1
speed_val = -1

dat %>%
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    filter( right_safety == safety_val | left_safety == safety_val,
            right_speed == speed_val | left_speed == speed_val) %>%
    mutate(selected_right = chosen_door_is_unknown == FALSE) %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )


print("Safety, Speed = [-1, 1]")
safety_val = -1
speed_val = 1

dat %>%
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    filter( right_safety == safety_val | left_safety == safety_val,
            right_speed == speed_val | left_speed == speed_val) %>%
    mutate(selected_right = chosen_door_is_unknown == FALSE) %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )


print("Safety, Speed = [-1, -1]")
safety_val = -1
speed_val = -1

dat %>%
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    filter( right_safety == safety_val | left_safety == safety_val,
            right_speed == speed_val | left_speed == speed_val) %>%
    mutate(selected_right = chosen_door_is_unknown == FALSE) %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )

[1] "Safety, Speed = [1, 1]"


selected_right_rate_mean,se,lower_bound,upper_bound
0.97,0.01,0.95,0.99


[1] "Safety, Speed = [1, -1]"


selected_right_rate_mean,se,lower_bound,upper_bound
0.92,0.02,0.88,0.95


[1] "Safety, Speed = [-1, 1]"


selected_right_rate_mean,se,lower_bound,upper_bound
0.28,0.05,0.17,0.39


[1] "Safety, Speed = [-1, -1]"


selected_right_rate_mean,se,lower_bound,upper_bound
0.14,0.03,0.07,0.2


# Accordance rates of Corrected Safety Prioritized Model (CSfM)

Change those decisions where one (1) unknown door is involved. The cue order remains to be Safety > Speed. However, an Unknown door is (always) better than an Unsafe one.

In [224]:
# Copy TTB Safety
dat$chosen_ttb_safety_corrected <- dat$chosen_ttb_safety

## Make changes to those decisions where an unknown door is involved
dat[dat$unknown_door_involved & dat$left_safety == 1 , 'chosen_ttb_safety_corrected'] <- 'left'
dat[dat$unknown_door_involved & dat$left_safety == -1 , 'chosen_ttb_safety_corrected'] <- 'right'

dat[dat$unknown_door_involved & dat$right_safety == 1 , 'chosen_ttb_safety_corrected'] <- 'right'
dat[dat$unknown_door_involved & dat$right_safety == -1 , 'chosen_ttb_safety_corrected'] <- 'left'

### Corrected Safety: Overall accordance rate

In [225]:
dat %>%
    filter( equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_safety )

dat %>%
    filter( equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_speed )

dat %>%
    filter( equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_safety_corrected )

accordance_rate_mean,se,lower_bound,upper_bound
0.73,0.01,0.7,0.75


accordance_rate_mean,se,lower_bound,upper_bound
0.66,0.01,0.64,0.69


accordance_rate_mean,se,lower_bound,upper_bound
0.89,0.01,0.86,0.92


### 4. Corrected Safety: Unknown door is involved

In [226]:
print('Safety')
dat %>% 
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_safety )

print('Speed')
dat %>% 
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_speed )

print('Corrected Safety')
dat %>% 
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    accordance( chosen_ttb_safety_corrected )

[1] "Safety"


accordance_rate_mean,se,lower_bound,upper_bound
0.58,0.02,0.53,0.62


[1] "Speed"


accordance_rate_mean,se,lower_bound,upper_bound
0.58,0.02,0.53,0.62


[1] "Corrected Safety"


accordance_rate_mean,se,lower_bound,upper_bound
0.87,0.02,0.83,0.91


### 4.2 Corrected Safety: Unknown door is involved. Digging deeper

In [227]:
print("Safety, Speed = [1, 1]")
safety_val = 1
speed_val = 1

dat %>%
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    filter( right_safety == safety_val | left_safety == safety_val,
            right_speed == speed_val | left_speed == speed_val) %>%
    mutate(selected_right = chosen_door_is_unknown == FALSE) %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )


print("Safety, Speed = [1, -1]")
safety_val = 1
speed_val = -1

dat %>%
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    filter( right_safety == safety_val | left_safety == safety_val,
            right_speed == speed_val | left_speed == speed_val) %>%
    mutate(selected_right = chosen_door_is_unknown == FALSE) %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )


print("Safety, Speed = [-1, 1]")
safety_val = -1
speed_val = 1

dat %>%
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    filter( right_safety == safety_val | left_safety == safety_val,
            right_speed == speed_val | left_speed == speed_val) %>%
    mutate(selected_right = chosen_door_is_unknown == FALSE) %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )


print("Safety, Speed = [-1, -1]")
safety_val = -1
speed_val = -1

dat %>%
    filter( unknown_door_involved == TRUE ,
            equivalent_options == FALSE ) %>%
    filter( right_safety == safety_val | left_safety == safety_val,
            right_speed == speed_val | left_speed == speed_val) %>%
    mutate(selected_right = chosen_door_is_unknown == FALSE) %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )

[1] "Safety, Speed = [1, 1]"


selected_right_rate_mean,se,lower_bound,upper_bound
0.97,0.01,0.95,0.99


[1] "Safety, Speed = [1, -1]"


selected_right_rate_mean,se,lower_bound,upper_bound
0.92,0.02,0.88,0.95


[1] "Safety, Speed = [-1, 1]"


selected_right_rate_mean,se,lower_bound,upper_bound
0.28,0.05,0.17,0.39


[1] "Safety, Speed = [-1, -1]"


selected_right_rate_mean,se,lower_bound,upper_bound
0.14,0.03,0.07,0.2


# Response times

### Overall response time per block
It is calculated as an average of averages.

In [228]:
response_times <- dat %>%
                    group_by(id, block) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by(block) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(x = block)

response_times

block,y,lower_bound,upper_bound,x
0,2669.345,2116.211,3222.479,0
1,1946.088,1592.342,2299.835,1
2,1677.167,1343.703,2010.630,2
3,1502.650,1298.696,1706.604,3
4,1639.612,1277.989,2001.234,4
5,1526.050,1230.720,1821.380,5


In [229]:
#p1

## Hypotheses comparison

TTB_Safety vs TTB_Speed

In [230]:
bar_graph_2 <- function(plot_data, title) {
    plot_data %>%
        ggplot(aes(x=factor(x), y=y, group=group, fill=factor(group)))+
        geom_bar(stat="identity", position=position_dodge()) +
        geom_errorbar(aes(ymin=lower_bound, ymax=upper_bound),
                      position=position_dodge(.9),
                      width=0.2 ) +
        labs(fill = "Cues checked") + xlab("Block") + ylab("Response time (ms.)") + ggtitle(title)
}

In [231]:
rt_ttb_safety <- dat %>%
                    group_by(id, block, steps_ttb_safety) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by(block, steps_ttb_safety) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(x = block + 1,
                           group = steps_ttb_safety)

p1 <- bar_graph_2(rt_ttb_safety, "(a)")

In [232]:
rt_ttb_safety_f <- dat %>%
                    filter(unknown_door_involved == FALSE) %>% # Unknown doors filtered out
                    group_by(id, block, steps_ttb_safety) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by(block, steps_ttb_safety) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(x = block + 1,
                           group = steps_ttb_safety)

p12 <- bar_graph_2(rt_ttb_safety_f, "(c)")

In [233]:
rt_ttb_speed <- dat %>%
                    group_by(id, block, steps_ttb_speed) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by(block, steps_ttb_speed) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(x = block + 1,
                           group = steps_ttb_speed)

p2 <- bar_graph_2(rt_ttb_speed, "(b)")

In [234]:
rt_ttb_speed <- dat %>%
                    filter(unknown_door_involved == FALSE) %>% # Unknown doors filtered out
                    group_by(id, block, steps_ttb_speed) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by(block, steps_ttb_speed) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(x = block + 1,
                           group = steps_ttb_speed)

p22 <- bar_graph_2(rt_ttb_speed, "(d)")

In [236]:
#svg("Images/response_times.svg", width=8, height=4)
#multiplot(p1, p12, p2, p22, cols=2)
#dev.off()

### Only known doors: response times

In [237]:
response_times <- dat %>%
                    filter(unknown_door_involved == FALSE) %>%
                    group_by(id, block) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by(block) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(x = block)

response_times

block,y,lower_bound,upper_bound,x
0,2508.158,1922.645,3093.671,0
1,1831.096,1474.993,2187.198,1
2,1488.537,1283.602,1693.473,2
3,1309.483,1128.594,1490.373,3
4,1537.992,1184.626,1891.358,4
5,1319.933,1109.113,1530.754,5


In [238]:
p2 = bar_graph(response_times)

### Individual response times

In [239]:
response_times <- dat %>%
                    group_by(id) %>%
                    summarize(y = mean(rt),
                              lower_bound = t.test(rt, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(rt, conf.level=0.95)$conf.int[2]) %>%
                    mutate(x=id)

head(response_times)

id,y,lower_bound,upper_bound,x
1,1271.289,1162.377,1380.201,1
10,1646.694,1443.289,1850.100,10
11,1531.006,1337.407,1724.604,11
12,1385.678,1274.645,1496.710,12
13,1118.083,1011.037,1225.130,13
14,2123.867,1798.781,2448.952,14


In [240]:
response_times %>%
    summarize(mean_rt = mean(y),
              lower_bound = t.test(y, conf.level=0.95)$conf.int[1],
              upper_bound = t.test(y, conf.level=0.95)$conf.int[2])

mean_rt,lower_bound,upper_bound
1826.819,1511.135,2142.502


#### Separated between those decisions that involved unknown doors and those that did not

In [241]:
response_times <- dat %>%
                    filter(id==1 | id==2 | id==4) %>%
                    group_by(id, unknown_door_involved) %>%
                    summarize(y = mean(rt),
                              se = se(rt))  %>%
                    mutate(lower_bound = y - se/2, 
                           upper_bound = y + se/2, 
                           x=paste(id, unknown_door_involved))

head(response_times)

id,unknown_door_involved,y,se,lower_bound,upper_bound,x
1,FALSE,1170.347,78.77117,1130.962,1209.733,1 FALSE
1,TRUE,1338.583,75.11780,1301.024,1376.142,1 TRUE
2,FALSE,1517.417,136.02304,1449.405,1585.428,2 FALSE
2,TRUE,1719.407,109.75263,1664.531,1774.284,2 TRUE
4,FALSE,1478.333,77.39547,1439.636,1517.031,4 FALSE
4,TRUE,1541.454,69.33742,1506.785,1576.122,4 TRUE


# Human versus ACT-R responses

### Only first block (overall)

In [293]:
rt_ttb_safety %>% filter(block==0)

block,steps_ttb_safety,y,lower_bound,upper_bound,x,group
0,1,2637.992,2055.129,3220.856,1,1
0,2,2873.137,2282.377,3463.898,1,2


### Only first block (only known doors)

In [405]:
print("Humans")
rt_ttb_safety_f %>% filter(block==5)

print("ACT-R")
agents %>%
                    filter(unknown_door_involved == FALSE) %>% # Unknown doors filtered out
                    group_by(id, steps_ttb_safety) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by(steps_ttb_safety) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(y = round(y, 3) * 1000,
                           lower_bound = round(lower_bound, 3) * 1000,
                           upper_bound = round(upper_bound, 3) * 1000)

[1] "Humans"


block,steps_ttb_safety,y,lower_bound,upper_bound,x,group
5,1,1167.394,926.9881,1407.799,6,1
5,2,1625.013,1340.8527,1909.172,6,2


[1] "ACT-R"


steps_ttb_safety,y,lower_bound,upper_bound
1,1258,1208,1309
2,1664,1624,1705


In [398]:
print("Humans")
dat %>% 
    filter( unknown_door_involved == FALSE,
            steps_ttb_safety == 1) %>%
    filter( block == 5 ) %>%
    accordance( chosen_ttb_safety_corrected )

dat %>% 
    filter( unknown_door_involved == FALSE,
            steps_ttb_safety == 2) %>%
    filter( block == 5 ) %>%
    accordance( chosen_ttb_safety_corrected )

print("ACT-R")
agents %>% 
    filter( unknown_door_involved == FALSE,
            steps_ttb_safety == 1) %>%
    accordance_actr( csfm )

agents %>% 
    filter( unknown_door_involved == FALSE,
            steps_ttb_safety == 2) %>%
    accordance_actr( csfm )

[1] "Humans"


accordance_rate_mean,se,lower_bound,upper_bound
0.95,0.02,0.9,1


accordance_rate_mean,se,lower_bound,upper_bound
0.9,0.04,0.82,0.98


[1] "ACT-R"


accordance_rate_mean,se,lower_bound,upper_bound
0.97,0.01,0.95,0.99


accordance_rate_mean,se,lower_bound,upper_bound
0.96,0.01,0.94,0.98


### Only first block (known vs unknown door)

In [406]:
print('Humans')

rt_ttb_safety_f2 <- dat %>%
                    filter(unknown_door_involved == TRUE) %>% # Unknown doors filtered IN
                    filter(equivalent_options == FALSE) %>% # But different
                    group_by(id, block, steps_ttb_safety) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by(block, steps_ttb_safety) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(x = block + 1,
                           group = steps_ttb_safety)

rt_ttb_safety_f2 %>% filter(block == 5)

print('ACT-R')

agents %>%
                    filter(unknown_door_involved == TRUE) %>% # Unknown doors filtered IN
                    filter(equivalent_options == FALSE) %>% # But different
                    group_by(id, steps_ttb_safety) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by(steps_ttb_safety) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(y = round(y, 3) * 1000,
                           lower_bound = round(lower_bound, 3) * 1000,
                           upper_bound = round(upper_bound, 3) * 1000)

[1] "Humans"


block,steps_ttb_safety,y,lower_bound,upper_bound,x,group
5,1,1557.372,1187.212,1927.532,6,1


[1] "ACT-R"


steps_ttb_safety,y,lower_bound,upper_bound
1,2010,1889,2131


In [400]:
print("Humans")
dat %>% 
    filter(unknown_door_involved == TRUE, # Unknown doors filtered IN
           equivalent_options == FALSE) %>% # But different
    filter( block == 5 ) %>%
    accordance( chosen_ttb_safety_corrected )

print("ACT-R")
agents %>% 
    filter(unknown_door_involved == TRUE, # Unknown doors filtered IN
           equivalent_options == FALSE) %>% # But different
    accordance_actr( csfm )

[1] "Humans"


accordance_rate_mean,se,lower_bound,upper_bound
0.82,0.04,0.74,0.91


[1] "ACT-R"


accordance_rate_mean,se,lower_bound,upper_bound
0.83,0.01,0.8,0.86


### Only first block (unknown vs unknown door)

In [407]:
print("Humans")
rt_ttb_safety_f3 <- dat %>%
                    filter(unknown_door_involved == TRUE) %>% # Unknown doors filtered IN
                    filter(equivalent_options == TRUE) %>%
                    group_by(id, block, steps_ttb_safety) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by(block, steps_ttb_safety) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(x = block + 1,
                           group = steps_ttb_safety)

rt_ttb_safety_f3 %>% filter(block == 5)

print("ACT-R")
agents %>%
                    filter(unknown_door_involved == TRUE) %>% # Unknown doors filtered IN
                    filter(equivalent_options == TRUE) %>%
                    group_by(id, steps_ttb_safety) %>%
                    summarize(subject_rt_mean = mean(rt)) %>% # per subject per block
                    group_by( steps_ttb_safety) %>%
                    summarize(y = mean(subject_rt_mean),
                              lower_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[1],
                              upper_bound = t.test(subject_rt_mean, conf.level=0.95)$conf.int[2]) %>%
                    mutate(y = round(y, 3) * 1000,
                           lower_bound = round(lower_bound, 3) * 1000,
                           upper_bound = round(upper_bound, 3) * 1000)

[1] "Humans"


block,steps_ttb_safety,y,lower_bound,upper_bound,x,group
5,1,2512.175,1612.958,3411.392,6,1


[1] "ACT-R"


steps_ttb_safety,y,lower_bound,upper_bound
1,2862,2688,3036


In [403]:
print("Humans")
dat %>%
    filter( unknown_door_involved == TRUE, # Unknown doors filtered IN
            equivalent_options == TRUE ) %>%
    filter( block == 5 ) %>%
    mutate(selected_right = key_pressed == 'right') %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )
    


print("ACT-R")
agents %>% 
    filter( unknown_door_involved == TRUE, # Unknown doors filtered IN
            equivalent_options == TRUE ) %>%
    mutate(selected_right = response == 'j') %>%
    group_by( id ) %>%
    summarize( selected_right_rate = mean(selected_right) ) %>%
    summarize( selected_right_rate_mean = mean(selected_right_rate), 
               se = se(selected_right_rate),
               lower_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[1],
               upper_bound = t.test(selected_right_rate, conf.level=0.95)$conf.int[2]) %>%
    round( decimals )

[1] "Humans"


selected_right_rate_mean,se,lower_bound,upper_bound
0.48,0.06,0.36,0.59


[1] "ACT-R"


selected_right_rate_mean,se,lower_bound,upper_bound
0.46,0.03,0.4,0.51


# Threshold analysis

In [132]:
agents_t <- read_csv("../ACT-R/outputs/experiment-threshold.csv", col_types = list(id = col_integer(), opt1 = col_character(), opt2 = col_character(), response = col_character(), time = col_double(), threshold = col_character()) )

# Response times
agents_t$rt <- agents_t$time - c(0, agents_t$time[1 : length(agents_t$time) - 1])
agents_t[agents_t$rt < 0, 'rt'] <- agents_t[agents_t$rt < 0, 'time']

# Chosen door
agents_t$chosen_door <- ifelse(agents_t$response == 'f', agents_t$opt1, agents_t$opt2)

#################

# Corrected TTB Safety (CSfM predictions)
agents_t$csfm <- "unknown"

# A is always superior
agents_t$csfm <- ifelse(agents_t$opt1 == "a" | agents_t$opt2 == "a", "a", agents_t$csfm)

# Option B
agents_t$csfm <- ifelse(agents_t$csfm == "unknown" & (agents_t$opt1 == "b" | agents_t$opt2 == "b"), "b", agents_t$csfm)

# C vs D, wins C
agents_t$csfm <- ifelse(agents_t$opt1 == "c" & agents_t$opt2 == "d", "c", agents_t$csfm)
agents_t$csfm <- ifelse(agents_t$opt1 == "d" & agents_t$opt2 == "c", "c", agents_t$csfm)

# C or D versus unknown
agents_t$csfm <- ifelse(agents_t$csfm == "unknown" & (agents_t$opt1 == "c" | agents_t$opt1 == "d"), agents_t$opt2, agents_t$csfm)
agents_t$csfm <- ifelse(agents_t$csfm == "unknown" & (agents_t$opt2 == "c" | agents_t$opt2 == "d"), agents_t$opt1, agents_t$csfm)

# Unknown door involved
agents_t$unknown_door_involved <- ( (agents_t$opt1 != 'a' & agents_t$opt1 != 'b' & agents_t$opt1 != 'c' & agents_t$opt1 != 'd') | 
                                  (agents_t$opt2 != 'a' & agents_t$opt2 != 'b' & agents_t$opt2 != 'c' & agents_t$opt2 != 'd') )

# Equivalent alternatives
agents_t$equivalent_options <- agents_t$csfm == "unknown"

# STEPS:
## Default is 1
agents_t$steps_ttb_safety <- 1

## 2 steps

### A vs B
agents_t$steps_ttb_safety <- ifelse((agents_t$opt1 == "a" & agents_t$opt2 == "b") |
                                  (agents_t$opt1 == "b" & agents_t$opt2 == "a"), 2, agents_t$steps_ttb_safety)

### C vs D
agents_t$steps_ttb_safety <- ifelse((agents_t$opt1 == "c" & agents_t$opt2 == "d") |
                                  (agents_t$opt1 == "d" & agents_t$opt2 == "c"), 2, agents_t$steps_ttb_safety)

In [133]:
agents_t$threshold <- ifelse(agents_t$threshold == '-10' |
                             agents_t$threshold == '-9' |
                             agents_t$threshold == '-8' |
                             agents_t$threshold == '-7' |
                             agents_t$threshold == '-6' |
                             agents_t$threshold == '-5' |
                             agents_t$threshold == '-4' |
                             agents_t$threshold == '-3' |
                             agents_t$threshold == '-2' |
                             agents_t$threshold == '-1' |
                             agents_t$threshold == '0' |
                             agents_t$threshold == '1' | 
                             agents_t$threshold == '2' |
                             agents_t$threshold == '3' |
                             agents_t$threshold == '4' |
                             agents_t$threshold == '5' |
                             agents_t$threshold == '6' |
                             agents_t$threshold == '7' |
                             agents_t$threshold == '8' |
                             agents_t$threshold == '9' |
                             agents_t$threshold == '10', paste0(agents_t$threshold, '/1'), agents_t$threshold)

agents_t$threshold_f <- sapply(strsplit(agents_t$threshold, '/'), function(x) as.numeric(x[1]) / as.numeric(x[2]))

### Retrieval threshold and response time

In [113]:
threshold_data <- agents_t %>%
    filter( unknown_door_involved == FALSE ) %>%
    group_by( id, threshold_f ) %>%
    summarize( rt_subject_mean = mean(rt) ) %>%
    group_by( threshold_f ) %>%
    summarize( rt_mean = mean(rt_subject_mean) * 1000, 
                   lower_bound = t.test(rt_subject_mean, conf.level=0.95)$conf.int[1] * 1000,
                   upper_bound = t.test(rt_subject_mean, conf.level=0.95)$conf.int[2] * 1000) %>%
        round( decimals )

In [159]:
threshold_graph <- ggplot(data=threshold_data, aes(x=threshold_f, y=rt_mean, group=1)) +
    geom_line() +
    geom_point() +
    expand_limits(y = 0) +
    scale_x_continuous(lim = c(-2, 7), breaks = seq(-2, 7, 1)) +
    geom_vline(xintercept = 0) +
    geom_ribbon(aes(ymin=lower_bound, ymax=upper_bound),alpha=0.3) +
    xlab("Retrieval threshold") + ylab("Average response time (ms.)")

svg("Images/threshold_graph.svg", width=8, height=4)
threshold_graph
dev.off()

Warning message:
"Removed 110 rows containing missing values (geom_path)."Warning message:
"Removed 110 rows containing missing values (geom_point)."

png 
  2

### Retrieval threshold and accordance rate

In [135]:
agents_t %>% head

id,opt1,opt2,response,time,threshold,rt,chosen_door,csfm,unknown_door_involved,equivalent_options,steps_ttb_safety,threshold_f
1,b,d,f,1.481,-10/1,1.481,b,b,FALSE,FALSE,1,-10
1,c,d,f,3.288,-10/1,1.807,c,c,FALSE,FALSE,2,-10
1,c,b,j,4.528,-10/1,1.240,b,b,FALSE,FALSE,1,-10
1,b,a,j,6.456,-10/1,1.928,a,a,FALSE,FALSE,2,-10
1,d,c,j,8.187,-10/1,1.731,c,c,FALSE,FALSE,2,-10
1,a,c,f,9.423,-10/1,1.236,a,a,FALSE,FALSE,1,-10


In [150]:
threshold_accordance_data <- agents_t %>%
    mutate( correctly_predicted = (chosen_door == csfm) ) %>%
    filter( unknown_door_involved == FALSE ) %>%
    group_by( id, threshold_f ) %>%
    summarize( accordance_rate = jitter(mean(correctly_predicted), 0.001) ) %>%
    group_by( threshold_f ) %>%
    summarize( accordance_rate_mean = mean(accordance_rate),
                   lower_bound = t.test(accordance_rate, conf.level=0.95)$conf.int[1],
                   upper_bound = t.test(accordance_rate, conf.level=0.95)$conf.int[2],
                   se = se(accordance_rate)) %>%
    round( decimals )

In [160]:
threshold_accordance_graph <- ggplot(data=threshold_accordance_data, aes(x=threshold_f, y=accordance_rate_mean, group=1)) +
    geom_line() +
    geom_point() +
    expand_limits(y = 0) +
    scale_x_continuous(lim = c(-2, 7), breaks = seq(-2, 7, 1)) +
    geom_vline(xintercept = 0) +
    geom_ribbon(aes(ymin=lower_bound, ymax=upper_bound),alpha=0.3) +
    xlab("Retrieval threshold") + ylab("Average accordance rate")

svg("Images/threshold_accordance_graph.svg", width=8, height=4)
threshold_accordance_graph
dev.off()

Warning message:
"Removed 110 rows containing missing values (geom_path)."Warning message:
"Removed 110 rows containing missing values (geom_point)."

png 
  2

# Explore

In [ ]:
dat %>%
    select(-KEY_RESPONSE_TIME, -DISPLAY_TIME, -row_id, -unknown_door_involved) %>%
    filter(chosen_ttb_speed != chosen_ttb_safety) %>%
    head

In [44]:
dat %>%
    filter(unknown_door_involved == TRUE) %>%
    select(-KEY_RESPONSE_TIME, -DISPLAY_TIME, -row_id, -unknown_door_involved) %>%
    filter(steps_ttb_safety == 1)

left_door,right_door,rt,key_pressed,block_level_1_fc,block,id,experiment,chosen_door,left_speed,...,right_speed,right_safety,chosen_speed,chosen_safety,steps_ttb_speed,steps_ttb_safety,chosen_door_is_unknown,chosen_ttb_speed,chosen_ttb_safety,equivalent_options
door005,door000,2271,left,1,0,1,A,door005,-1,...,0,0,-1,1,1,1,FALSE,left,left,FALSE
door004,door000,567,left,1,0,1,A,door004,1,...,0,0,1,1,1,1,FALSE,left,left,FALSE
door001,door000,3496,right,1,0,1,A,door000,1,...,0,0,0,0,1,1,TRUE,left,left,FALSE
door000,door004,1084,right,1,0,1,A,door004,0,...,1,1,1,1,1,1,FALSE,right,right,FALSE
door002,door003,4220,left,1,0,1,A,door002,0,...,-1,-1,0,0,1,1,TRUE,right,right,FALSE
door001,door002,1124,right,1,0,1,A,door002,1,...,0,0,0,0,1,1,TRUE,left,left,FALSE
door003,door002,1244,right,1,0,1,A,door002,-1,...,0,0,0,0,1,1,TRUE,left,left,FALSE
door002,door001,2411,left,1,0,1,A,door002,0,...,1,-1,0,0,1,1,TRUE,right,right,FALSE
door002,door005,1844,right,1,0,1,A,door005,0,...,-1,1,-1,1,1,1,FALSE,right,right,FALSE
door003,door000,1547,left,1,0,1,A,door003,-1,...,0,0,-1,-1,1,1,FALSE,left,left,FALSE
